# Resimulate outbreaks

This notebook contains archived code that was written to take the regression coefficients estimated in `notebooks/simulate-and-regress.ipynb` and re-simulate the outbreak using SIR (as in the main text Fig. 4) and/or SEIR (as in Extended Data Fig. 7) dynamics. It is not finished and some of the input data structures have changed, so please do not use this without vetting and updating it first (and please submit a Pull Request when you do!)

In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import xarray as xr
from src import utils as cutil
from src.models import epi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Settings

In [5]:
reg_dir = cutil.RESULTS / "other" / "sims" / "measNoise_0.1_betaNoise_Exp_gammaNoise_0.01_sigmaNoise_0.03"

## Regenerate predictions of cumulative cases under no-policy and policy scenarios

In [15]:
coeffs = epi.load_and_combine_reg_results(
    reg_dir, cols_to_keep=["effect", "Intercept", "beta_deterministic", "IR"] + list("SIR")
)
coeffs = epi.calc_cum_effects(coeffs)

### True no-policy counterfactual

In [ ]:
def run_sim_across_pops(proj_ds, reg_res, kind):
    proj_out = []
    this_ds = proj_ds
    for p in reg_res.pop:
        if "pop" in proj_ds.dims:
            this_ds = proj_ds.sel(pop=p)
        if kind=="SEIR":
            this_out = epi.run_SEIR(
                reg_res.attrs["E0"] / p,
                reg_res.attrs["I0"] / p,
                reg_res.attrs["R0"] / p,
                this_ds,
            )
        elif kind=="SIR":
            this_out = epi.run_SIR(
                reg_res.attrs["E0"] / p,
                reg_res.attrs["R0"] / p,
                this_ds,
            )
        else:
            raise ValueError
        proj_out.append(this_out)
    out = xr.concat(proj_out, dim="pop")
    out["pop"] = reg_res.pop.values
    out["t"] = out.t.round(5)
    out = out.reindex(t=reg_res.t.astype(int))
    out.attrs = reg_res.attrs
    return out

In [ ]:
tsteps_per_day = reg_res.attrs["tsteps_per_day"]
t = np.arange(0, reg_res.t.max() + 1 / tsteps_per_day, 1 / tsteps_per_day)
t = xr.DataArray(coords={"t": t}, dims=["t",], data=t)
beta_np_true = epi.get_beta_SEIR(
    xr.ones_like(t) * reg_res.attrs["no_policy_growth_rate"],
    reg_res.gamma,
    reg_res.sigma,
)
proj_ds = beta_np_true.to_dataset(name="beta_deterministic")

proj_ds = epi.get_stochastic_params(
    proj_ds,
    reg_res.attrs["beta_noise_sd"],
    bool(reg_res.attrs["beta_noise_on"]),
    reg_res.attrs["gamma_noise_sd"],
    bool(reg_res.attrs["gamma_noise_on"]),
    sigma_noise_sd=reg_res.attrs["sigma_noise_sd"],
    sigma_noise_on=bool(reg_res.attrs["sigma_noise_on"]),
)

proj_ds = epi.adjust_timescales_from_daily(proj_ds)

# # Run simulation
true_np = run_sim_across_pops(proj_ds, reg_res, "SEIR")

### Predicted states

First, we estimate gamma from the data

In [113]:
dRdt = reg_res.R.diff("t", 1)
I = ((reg_res.I.shift(t=1) + reg_res.I) / 2).isel(t=slice(1, None))
gamma_est_ds = (dRdt / I).mean(dim="t")

Then, we get our estimates of the uninhibited exponential growth rate w/ and w/o policy.

In [114]:
def get_lambda(reg_res, lagged_policy, proj_ds):
    lambda_pred_p = reg_res.Intercept.copy().broadcast_like(proj_ds)
    lambda_pred_p = lambda_pred_p.transpose("sample","t","gamma","sigma","LHS","pop")
    tmp = lambda_pred_p.values.copy()

    p_on = (reg_res.policy_timeseries > 0).argmax(dim="t").isel(pop=0).squeeze()
    for p in reg_res.policy.values:
        for l in reg_res.reg_lag.values:
            lvar = reg_res.coefficient.sel(policy=p, reg_lag=l).transpose("sample","gamma","sigma","LHS","pop").values
            for s in reg_res.sample.values:
                this_on = (
                    p_on.sel(sample=s, policy=p).item() * reg_res.attrs["tsteps_per_day"]
                )
                if lagged_policy:
                    this_on += l * reg_res.attrs["tsteps_per_day"]
                tmp[s,this_on:] += lvar[s]
    lambda_pred_p.values = tmp
    return lambda_pred_p

lambda_pred_p_SIR = get_lambda(reg_res, True, proj_ds)
lambda_pred_p_SEIR = get_lambda(reg_res, False, proj_ds)
lambda_pred_np = reg_res.Intercept.broadcast_like(lambda_pred_p_SEIR)

Then we run an SEIR and SIR model

In [116]:
def get_cum_cases_from_seir(lambda_pred, gamma_est_ds, reg_res):
    # SEIR
    beta_np_true = epi.get_beta_SEIR(
        lambda_pred,
        gamma_est_ds,
        lambda_pred.sigma.rename({"sigma": "sigma_proj"}),
    )
    proj_ds = beta_np_true.to_dataset(name="beta_deterministic")
    proj_ds["gamma_deterministic"] = gamma_est_ds

    proj_ds = epi.get_stochastic_params(
        proj_ds,
        reg_res.attrs["beta_noise_sd"],
        bool(reg_res.attrs["beta_noise_on"]),
        reg_res.attrs["gamma_noise_sd"],
        bool(reg_res.attrs["gamma_noise_on"]),
        sigma_noise_sd=reg_res.attrs["sigma_noise_sd"],
        sigma_noise_on=bool(reg_res.attrs["sigma_noise_on"]),
    )

    proj_ds = epi.adjust_timescales_from_daily(proj_ds)

    # Run simulation
    true_np_seir = run_sim_across_pops(proj_ds, reg_res, "SEIR")
    return true_np_seir

def get_cum_cases_from_sir(lambda_pred, gamma_est_ds, reg_res):
    # SIR
    proj_ds = (lambda_pred + gamma_est_ds).to_dataset(name="beta_deterministic")
    proj_ds["gamma_deterministic"] = gamma_est_ds
    proj_ds = epi.get_stochastic_params(
        proj_ds,
        reg_res.attrs["beta_noise_sd"],
        bool(reg_res.attrs["beta_noise_on"]),
        reg_res.attrs["gamma_noise_sd"],
        bool(reg_res.attrs["gamma_noise_on"]),
    ).drop("sigma_stoch")
    proj_ds = epi.adjust_timescales_from_daily(proj_ds)

    # Run simulation
    true_np_sir = run_sim_across_pops(proj_ds, reg_res, "SIR")
    return true_np_sir

Now we calculate final system state with and without policy

In [117]:
pred_p_SIR = get_cum_cases_from_sir(lambda_pred_p_SIR, gamma_est_ds, reg_res)
pred_np_SIR = get_cum_cases_from_sir(lambda_pred_np, gamma_est_ds, reg_res)
pred_p_SEIR = get_cum_cases_from_seir(lambda_pred_p_SEIR, gamma_est_ds, reg_res)
pred_np_SEIR = get_cum_cases_from_seir(lambda_pred_np, gamma_est_ds, reg_res)

Now we aggregate all of the true and pred values

In [119]:
sim_dir_base = out_home / "projection"
bn, gn, sn = (0, 0, 0)
if beta_noise_on:
    bn = reg_res.attrs["beta_noise_sd"]
if gamma_noise_on:
    gn = reg_res.attrs["gamma_noise_sd"]
if sigma_noise_on:
    sn = reg_res.attrs["sigma_noise_sd"]
    
sim_dir = sim_dir_base / f"betaNoise_{bn}_gammaNoise_{gn}_sigmaNoise_{sn}"
sim_dir.mkdir(parents=True, exist_ok=True)

In [137]:
sir_pred = xr.concat([pred_np_SIR[list("SIR")+["beta_deterministic", "gamma_deterministic"]], pred_p_SIR[list("SIR")+["beta_deterministic", "gamma_deterministic"]]], dim="with_policy")
sir_pred.to_netcdf(sim_dir / "pred_sir.nc")
seir_pred = xr.concat([pred_np_SEIR[list("SEIR")+["beta_deterministic", "gamma_deterministic"]], pred_p_SEIR[list("SEIR")+["beta_deterministic", "gamma_deterministic"]]], dim="with_policy")
seir_pred.to_netcdf(sim_dir / "pred_seir.nc")
true = xr.concat([true_np[list("SEIR")+["beta_deterministic"]], true_p[list("SEIR")+["beta_deterministic"]]], dim="with_policy")
true.to_netcdf(sim_dir / "true.nc")